In [1]:
import sys
sys.path.append('/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/')
import time
from scipy import stats
from statsmodels.stats.diagnostic import lilliefors
import random
import os

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from nlcd_user import *

In [3]:
########## making table of non linearity using ks test ##########
genotype=pd.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/genotypes_binary_strongest_eqtl.csv",header=0,index_col=0)
expression_data=pd.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/expression_statsmodels_linreg_residuals_01.csv",index_col=0,header=0)
causal_conf=read_configuration("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/yeastgt_1_wilko1752_ready.txt")
indp_conf=read_configuration("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/yeastgt_0_wilko1752_ready.txt")


In [4]:
lineardata=read_data("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/data/Linear500.txt")
sinedata=read_data("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/data/Sine500.txt")
sawdata=read_data("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/data/Saw500.txt")
paradata=read_data("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/data/Para500.txt")

In [5]:
def calculate_ks_yeast(conf):
    total=[]
    for  i in range(len(conf)):
        trio=conf.iloc[i,]
        L_name=trio[0]
        A_name=trio[1]
        B_name=trio[2]
        L=genotype.loc[:,L_name]
        A=expression_data.loc[:,A_name]
        B=expression_data.loc[:,B_name]
        unique_values = np.unique(L)
        indv=[]
        for value in unique_values:
            indices = np.where(L == value)[0]
            A_value = A[indices]    
            indv.extend(lilliefors(A_value))
            #indv.append(stats.shapiro(A_value)[1])
        for value in unique_values:
            indices = np.where(L == value)[0]
            B_value = B[indices]    
            indv.extend(lilliefors(B_value))
        total.append(indv)
    ks_result=pd.DataFrame(total,columns=['A|L = 0 stat','A|L = 0 p','A|L = 1 stat','A|L = 1 p','B|L = 0 stat','B|L = 0 p','B|L = 1 stat','B|L = 1 p',])
    return ks_result

In [6]:
def calculate_ks_simdata(data):
    total=[]
    for  i in range(len(data)):
        L=data[i][0]
        A=data[i][1]
        B=data[i][2]
        unique_values = np.unique(L)
        indv=[]
        for value in unique_values:
            indices = np.where(L == value)[0]
            A_value = A[indices]    
            indv.extend(lilliefors(A_value))
        for value in unique_values:
            indices = np.where(L == value)[0]
            B_value = B[indices]    
            indv.extend(lilliefors(B_value))
        total.append(indv)
    ks_result=pd.DataFrame(total,columns=['A|L = 0 stat','A|L = 0 p','A|L = 1 stat','A|L = 1 p','B|L = 0 stat','B|L = 0 p','B|L = 1 stat','B|L = 1 p',])
    return ks_result

In [7]:
def cutoff_plot(ks_result):
    cutoffs=[0,0.01,0.02,0.03,0.05,0.06,0.07,0.08,0.09,0.1]
    A_passed=[]
    B_passed=[]
    both=[]
    for cutoff in cutoffs:
        ks_result['A_passed']= (ks_result['A|L = 0 p']>=cutoff)& (ks_result['A|L = 1 p']>=cutoff)
        ks_result['B_passed']=(ks_result['B|L = 0 p']>=cutoff) & (ks_result['B|L = 1 p']>=cutoff)
        ks_result['both']=ks_result['A_passed'] & ks_result['B_passed']
        A_passed.append(sum(ks_result['A_passed']))
        B_passed.append(sum(ks_result['B_passed']))
        both.append(sum(ks_result['both']))
    finaltable=pd.DataFrame({'cutoffs':cutoffs,'A passed':A_passed,'B_passed':B_passed,'both':both})
    return finaltable

In [8]:
yeasttable_indp=cutoff_plot(calculate_ks_yeast(indp_conf))
yeasttable_causal=cutoff_plot(calculate_ks_yeast(causal_conf))
linear_table=cutoff_plot(calculate_ks_simdata(lineardata))
sine_table=cutoff_plot(calculate_ks_simdata(sinedata))
saw_table=cutoff_plot(calculate_ks_simdata(sawdata))
para_table=cutoff_plot(calculate_ks_simdata(paradata))


In [9]:
para_table

,cutoffs,A passed,B_passed,both
0,0.00,100,100,100
1,0.01,98,14,14
2,0.02,97,13,13
3,0.03,97,13,13
4,0.05,93,10,10
5,0.06,93,9,9
6,0.07,92,9,9
7,0.08,88,9,9
8,0.09,87,9,9
9,0.10,85,9,9


In [9]:
ksresult=calculate_ks_simdata(paradata)

In [10]:
simresult=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/2way/SawKRR500s100perm.csv")
simresult_rev=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/2way/SawKRR500s100perm_rev.csv")

In [12]:
# # plots not required anymore
# simresult=pd.read_csv("./../../../conclude/newmethod/simulation_final/results/journal/simulation/nlcd/ParaKRR500s100perm.csv")
# fig, axs = plt.subplots(2,2)
# fig.set_figheight(8)
# fig.set_figwidth(8)
# fig.suptitle("Para data")
# axs[0,0].scatter(ksresult['B|L = 1 stat'],simresult['p_final']  )
# axs[0,0].set_xlabel("B|L = 1 liliferos statistic")
# axs[0,0].set_ylabel("p value A -> B")
# axs[0,1].scatter(ksresult['B|L = 0 stat'],simresult['p_final']  )
# axs[0,1].set_xlabel("B|L = 0 liliferos statistic")
# axs[0,1].set_ylabel("p value A -> B")
# axs[1,0].scatter(ksresult['B|L = 1 p'],simresult['p_final']  )
# axs[1,0].set_xlabel("B|L = 1 lilliferos p value")
# axs[1,0].set_ylabel("p value A -> B")
# axs[1,1].scatter(ksresult['B|L = 0 p'],simresult['p_final']  )
# axs[1,1].set_xlabel("B|L = 0 lilliferos p value")
# axs[1,1].set_ylabel("p value A -> B")

In [13]:
# fig, axs = plt.subplots(2, 2)
# fig.set_figheight(8)
# fig.set_figwidth(8)
# fig.suptitle("Para data")
# axs[0,0].scatter(ksresult['B|L = 1 stat'],simresult['p_final']  )
# axs[0,0].set_xlabel("B|L = 1 liliferos statistic")
# axs[0,0].set_ylabel("p value A -> B")
# axs[0,1].scatter(ksresult['B|L = 0 stat'],simresult['p_final']  )
# axs[0,1].set_xlabel("B|L = 0 liliferos statistic")
# axs[0,1].set_ylabel("p value A -> B")
# axs[1,0].scatter(ksresult['B|L = 1 stat'],simresult_rev['p_final']  )
# axs[1,0].set_xlabel("B|L = 1 liliferos statistic")
# axs[1,0].set_ylabel("p value B -> A")
# axs[1,1].scatter(ksresult['B|L = 0 stat'],simresult_rev['p_final']  )
# axs[1,1].set_xlabel("B|L = 0 liliferos statistic")
# axs[1,1].set_ylabel("p value B -> A")

In [14]:
# fig, axs = plt.subplots(2, 2)
# fig.set_figheight(8)
# fig.set_figwidth(8)
# fig.suptitle("Saw data")
# axs[0,0].scatter(ksresult['B|L = 1 p'],simresult['p_final']  )
# axs[0,0].set_xlabel("B|L = 1 lilliferos p value")
# axs[0,0].set_ylabel("p value A -> B")
# axs[0,1].scatter(ksresult['B|L = 0 p'],simresult['p_final']  )
# axs[0,1].set_xlabel("B|L = 0 lilliferos p value")
# axs[0,1].set_ylabel("p value A -> B")
# axs[1,0].scatter(ksresult['B|L = 1 p'],simresult_rev['p_final']  )
# axs[1,0].set_xlabel("B|L = 1 lilliferos p value")
# axs[1,0].set_ylabel("p value B -> A")
# axs[1,1].scatter(ksresult['B|L = 0 p'],simresult_rev['p_final']  )
# axs[1,1].set_xlabel("B|L = 0 lilliferos p value")
# axs[1,1].set_ylabel("p value B -> A")

In [15]:
# L=data[98][0]
# B=data[98][2]
# indices = np.where(L == 1)[0]
# B_value = B[indices]    
# plt.hist(B_value)
# lilliefors(B_value)

In [16]:
# cutoff=0.05
# ks_result['A_passed']= (ks_result['A|L = 0 p']>=cutoff)& (ks_result['A|L = 1 p']>=cutoff)
# ks_result['B_passed']=(ks_result['B|L = 0 p']>=cutoff) & (ks_result['B|L = 1 p']>=cutoff)
# ks_result['both']=ks_result['A_passed'] & ks_result['B_passed']
# ks_result

In [84]:
# simresult=pd.read_csv("./../../../conclude/newmethod/simulation_final/results/journal/2way/LinearKRR500s100perm.csv")

In [85]:
# simresult_rev=pd.read_csv("./../../../conclude/newmethod/simulation_final/results/journal/2way/LinearKRR500s100perm_rev.csv")

In [17]:
# lilliefors(x,dist='norm') # by default the distribution is normal 

In [18]:
# plt.hist(ks_result['B|L = 1 stat'])

## Calculating the number of trios that pass where liliferos test <0.05 and p value < 0.05 for the variable A

In [11]:
#make the confusion matrix 
#linear data 
from sklearn.metrics import confusion_matrix
df=calculate_ks_simdata(lineardata)
df_result=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/simulation/nlcd/LinearKRR500s100perm.csv")
df['A_final']=np.minimum(df['A|L = 0 p'],df['A|L = 1 p'])
print(confusion_matrix(df_result['p_final']<0.05,df['A_final']<0.05))
print('p value of A->B < 0.05 and liliferos test < 0.05 = ',sum((df_result['p_final']<0.05) & (df['A_final']<0.05)))
print('p value of A->B < 0.05 and liliferos test >= than 0.05 = ',sum((df_result['p_final']<0.05) & (df['A_final']>=0.05)))
print('p value of A->B >= 0.05 and liliferos test < than 0.05 = ' ,sum((df_result['p_final']>=0.05) & (df['A_final']<0.05)))
print('p value of A->B >= 0.05 and liliferos test  >= 0.05 = ',sum((df_result['p_final']>=0.05) & (df['A_final']>=0.05)))


[[23  0]
 [72  5]]
p value of A->B < 0.05 and liliferos test < 0.05 =  5
p value of A->B < 0.05 and liliferos test >= than 0.05 =  72
p value of A->B >= 0.05 and liliferos test < than 0.05 =  0
p value of A->B >= 0.05 and liliferos test  >= 0.05 =  23


In [12]:
#make the confusion matrix 
#sine data 
from sklearn.metrics import confusion_matrix
df=calculate_ks_simdata(sinedata)
df_result=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/simulation/nlcd/SineKRR500s100perm.csv")
df['A_final']=np.minimum(df['A|L = 0 p'],df['A|L = 1 p'])
print(confusion_matrix(df_result['p_final']<0.05,df['A_final']<0.05))
print('p value of A->B < 0.05 and liliferos test < 0.05 = ',sum((df_result['p_final']<0.05) & (df['A_final']<0.05)))
print('p value of A->B < 0.05 and liliferos test >= than 0.05 = ',sum((df_result['p_final']<0.05) & (df['A_final']>=0.05)))
print('p value of A->B >= 0.05 and liliferos test < than 0.05 = ' ,sum((df_result['p_final']>=0.05) & (df['A_final']<0.05)))
print('p value of A->B >= 0.05 and liliferos test  >= 0.05 = ',sum((df_result['p_final']>=0.05) & (df['A_final']>=0.05)))


[[58  6]
 [35  1]]
p value of A->B < 0.05 and liliferos test < 0.05 =  1
p value of A->B < 0.05 and liliferos test >= than 0.05 =  35
p value of A->B >= 0.05 and liliferos test < than 0.05 =  6
p value of A->B >= 0.05 and liliferos test  >= 0.05 =  58


In [13]:
#make the confusion matrix 
#saw data 
from sklearn.metrics import confusion_matrix
df=calculate_ks_simdata(sawdata)
df_result=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/simulation/nlcd/SawKRR500s100perm.csv")
df['A_final']=np.minimum(df['A|L = 0 p'],df['A|L = 1 p'])
print(confusion_matrix(df_result['p_final']<0.05,df['A_final']<0.05))
print('p value of A->B < 0.05 and liliferos test < 0.05 = ',sum((df_result['p_final']<0.05) & (df['A_final']<0.05)))
print('p value of A->B < 0.05 and liliferos test >= than 0.05 = ',sum((df_result['p_final']<0.05) & (df['A_final']>=0.05)))
print('p value of A->B >= 0.05 and liliferos test < than 0.05 = ' ,sum((df_result['p_final']>=0.05) & (df['A_final']<0.05)))
print('p value of A->B >= 0.05 and liliferos test  >= 0.05 = ',sum((df_result['p_final']>=0.05) & (df['A_final']>=0.05)))


[[72  7]
 [20  1]]
p value of A->B < 0.05 and liliferos test < 0.05 =  1
p value of A->B < 0.05 and liliferos test >= than 0.05 =  20
p value of A->B >= 0.05 and liliferos test < than 0.05 =  7
p value of A->B >= 0.05 and liliferos test  >= 0.05 =  72


In [14]:
#make the confusion matrix 
#parabola data 
from sklearn.metrics import confusion_matrix
df=calculate_ks_simdata(lineardata)
df_result=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/simulation/nlcd/ParaKRR500s100perm.csv")
df['A_final']=np.minimum(df['A|L = 0 p'],df['A|L = 1 p'])
print(confusion_matrix(df_result['p_final']<0.05,df['A_final']<0.05))
print('p value of A->B < 0.05 and liliferos test < 0.05 = ',sum((df_result['p_final']<0.05) & (df['A_final']<0.05)))
print('p value of A->B < 0.05 and liliferos test >= than 0.05 = ',sum((df_result['p_final']<0.05) & (df['A_final']>=0.05)))
print('p value of A->B >= 0.05 and liliferos test < than 0.05 = ' ,sum((df_result['p_final']>=0.05) & (df['A_final']<0.05)))
print('p value of A->B >= 0.05 and liliferos test  >= 0.05 = ',sum((df_result['p_final']>=0.05) & (df['A_final']>=0.05)))


[[19  1]
 [76  4]]
p value of A->B < 0.05 and liliferos test < 0.05 =  4
p value of A->B < 0.05 and liliferos test >= than 0.05 =  76
p value of A->B >= 0.05 and liliferos test < than 0.05 =  1
p value of A->B >= 0.05 and liliferos test  >= 0.05 =  19


## Calculating the number of trios that pass where liliferos test <0.05 and p value < 0.05 for the variable B

In [15]:
#make the confusion matrix 
#linear data 
from sklearn.metrics import confusion_matrix
df=calculate_ks_simdata(lineardata)
df_result=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/simulation/nlcd/LinearKRR500s100perm.csv")
df['B_final']=np.minimum(df['B|L = 0 p'],df['B|L = 1 p'])
print(confusion_matrix(df_result['p_final']<0.05,df['B_final']<0.05))
print('p value of A->B < 0.05 and liliferos test < 0.05 = ',sum((df_result['p_final']<0.05) & (df['B_final']<0.05)))
print('p value of A->B < 0.05 and liliferos test >= than 0.05 = ',sum((df_result['p_final']<0.05) & (df['B_final']>=0.05)))
print('p value of A->B >= 0.05 and liliferos test < than 0.05 = ' ,sum((df_result['p_final']>=0.05) & (df['B_final']<0.05)))
print('p value of A->B >= 0.05 and liliferos test  >= 0.05 = ',sum((df_result['p_final']>=0.05) & (df['B_final']>=0.05)))


[[22  1]
 [73  4]]
p value of A->B < 0.05 and liliferos test < 0.05 =  4
p value of A->B < 0.05 and liliferos test >= than 0.05 =  73
p value of A->B >= 0.05 and liliferos test < than 0.05 =  1
p value of A->B >= 0.05 and liliferos test  >= 0.05 =  22


In [16]:
#make the confusion matrix 
#sine data 
from sklearn.metrics import confusion_matrix
df=calculate_ks_simdata(sinedata)
df_result=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/simulation/nlcd/SineKRR500s100perm.csv")
df['B_final']=np.minimum(df['B|L = 0 p'],df['B|L = 1 p'])
print(confusion_matrix(df_result['p_final']<0.05,df['B_final']<0.05))
print('p value of A->B < 0.05 and liliferos test < 0.05 = ',sum((df_result['p_final']<0.05) & (df['B_final']<0.05)))
print('p value of A->B < 0.05 and liliferos test >= than 0.05 = ',sum((df_result['p_final']<0.05) & (df['B_final']>=0.05)))
print('p value of A->B >= 0.05 and liliferos test < than 0.05 = ' ,sum((df_result['p_final']>=0.05) & (df['B_final']<0.05)))
print('p value of A->B >= 0.05 and liliferos test  >= 0.05 = ',sum((df_result['p_final']>=0.05) & (df['B_final']>=0.05)))


[[37 27]
 [21 15]]
p value of A->B < 0.05 and liliferos test < 0.05 =  15
p value of A->B < 0.05 and liliferos test >= than 0.05 =  21
p value of A->B >= 0.05 and liliferos test < than 0.05 =  27
p value of A->B >= 0.05 and liliferos test  >= 0.05 =  37


In [17]:
#make the confusion matrix 
#saw data 
from sklearn.metrics import confusion_matrix
df=calculate_ks_simdata(sawdata)
df_result=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/simulation/nlcd/SawKRR500s100perm.csv")
df['B_final']=np.minimum(df['B|L = 0 p'],df['B|L = 1 p'])
print(confusion_matrix(df_result['p_final']<0.05,df['B_final']<0.05))
print('p value of A->B < 0.05 and liliferos test < 0.05 = ',sum((df_result['p_final']<0.05) & (df['B_final']<0.05)))
print('p value of A->B < 0.05 and liliferos test >= than 0.05 = ',sum((df_result['p_final']<0.05) & (df['B_final']>=0.05)))
print('p value of A->B >= 0.05 and liliferos test < than 0.05 = ' ,sum((df_result['p_final']>=0.05) & (df['B_final']<0.05)))
print('p value of A->B >= 0.05 and liliferos test  >= 0.05 = ',sum((df_result['p_final']>=0.05) & (df['B_final']>=0.05)))


[[65 14]
 [ 7 14]]
p value of A->B < 0.05 and liliferos test < 0.05 =  14
p value of A->B < 0.05 and liliferos test >= than 0.05 =  7
p value of A->B >= 0.05 and liliferos test < than 0.05 =  14
p value of A->B >= 0.05 and liliferos test  >= 0.05 =  65


In [18]:
#make the confusion matrix 
#parabola data 
from sklearn.metrics import confusion_matrix
df=calculate_ks_simdata(lineardata)
df_result=pd.read_csv("/data/users/cs20s037/CITNonLinear/conclude/newmethod/simulation_final/results/journal/simulation/nlcd/ParaKRR500s100perm.csv")
df['B_final']=np.minimum(df['B|L = 0 p'],df['B|L = 1 p'])
print(confusion_matrix(df_result['p_final']<0.05,df['B_final']<0.05))
print('p value of A->B < 0.05 and liliferos test < 0.05 = ',sum((df_result['p_final']<0.05) & (df['B_final']<0.05)))
print('p value of A->B < 0.05 and liliferos test >= than 0.05 = ',sum((df_result['p_final']<0.05) & (df['B_final']>=0.05)))
print('p value of A->B >= 0.05 and liliferos test < than 0.05 = ' ,sum((df_result['p_final']>=0.05) & (df['B_final']<0.05)))
print('p value of A->B >= 0.05 and liliferos test  >= 0.05 = ',sum((df_result['p_final']>=0.05) & (df['B_final']>=0.05)))


[[20  0]
 [75  5]]
p value of A->B < 0.05 and liliferos test < 0.05 =  5
p value of A->B < 0.05 and liliferos test >= than 0.05 =  75
p value of A->B >= 0.05 and liliferos test < than 0.05 =  0
p value of A->B >= 0.05 and liliferos test  >= 0.05 =  20


## The next cells are for sergregation of dataset based on variance of A which are further used in barplots 

In [19]:
commontfs=pd.read_csv('/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/commontfs.csv',header=None)[0].to_list()
df=pd.read_csv('/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/strongest_eqtls_r_3columns.csv',header=0)
df.drop(columns=df.columns[[0,3]], axis=1,  inplace=True)
df.columns=['cis','pmarker']
reqd=df[df['cis'].isin(commontfs)].reset_index()
reqd.drop(columns=reqd.columns[0],axis=1,inplace=True)

In [20]:
## read the genotype ## 
genotype=pd.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/genotypes_binary_strongest_eqtl.csv",header=0,index_col=0)

In [21]:
expression_data=pd.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/expression_statsmodels_linreg_residuals_01.csv",index_col=0,header=0)

In [22]:
reqd

,cis,pmarker
0,YBL005W,chrII:216631_A/G
1,YBR083W,chrII:408801_T/C
2,YBR239C,chrII:699675_T/C
3,YCR065W,chrIII:236451_G/A
4,YCR084C,chrIII:262264_T/C
...,...,...
75,YOR363C,chrXV:1023392_T/G
76,YPL038W,chrXVI:481979_C/T
77,YPL133C,chrXVI:310970_A/C
78,YPL248C,chrXVI:81157_G/A


In [23]:
#calculate the variance for each value of L 
var_list=[]
for i in range(len(reqd)):
    L=genotype.loc[:,reqd.loc[i,'pmarker']]
    A=expression_data.loc[:,reqd.loc[i,'cis']]
    unique_values = np.unique(L)
    variance = []

    for value in unique_values:
        indices = np.where(L == value)[0]
        A_value = A[indices]
        variance.append(np.var(A_value))
    var_list.append(variance)

In [24]:
reqd[['A_0','A_1']]=var_list
reqd['absdifference']=abs((reqd['A_0']-reqd['A_1'])/reqd['A_0'])

In [25]:
cutoffs=[0,0.05,0.1,0.125,0.15,0.175,0.2,0.225,0.25]
for i in cutoffs:
    varmore=reqd.loc[reqd['absdifference']>=i,]
    causal=read_configuration("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/yeastgt_1_wilko1752_ready.txt")
    causal=causal.reset_index()
    causal.columns=['id','L','A','B']
    merged_table = pd.merge(varmore,causal, left_on=['cis', 'pmarker'], right_on=['A', 'L'])
    print(len(merged_table))
    np.savetxt("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/var_indices/yeast_wilko_var_causal_"+str(i)+".csv",
        merged_table['id'],
        delimiter =", ",
        fmt ='% s')
    indep=read_configuration("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/yeastgt_0_wilko1752_ready.txt")
    indep=indep.reset_index()
    indep.columns=['id','L','A','B']
    merged_table = pd.merge(varmore,indep, left_on=['cis', 'pmarker'], right_on=['A', 'L'])
    print(len(merged_table))
    np.savetxt("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/var_indices/yeast_wilko_var_indep_"+str(i)+".csv",
        merged_table['id'],
        delimiter =", ",
        fmt ='% s')


1752
1752
1465
1465
1401
1401
1356
1356
623
623
581
581
574
574
516
516
297
297


## take the common ones among unequal variance and nonlinear (bcmi cutoffs)

In [ ]:
#var_cutoff=[0,0.1,0.125,0.15,0.175,0.2,0.225,0.25]
var_cutoff=[0,0.05,0.1,0.125,0.15]
bcmi_cutoff=[0.05,0.1,0.15]
odc_cutoff=[0.05]
seed_number=10 # change the seed here from 1 to 10
os.chdir('/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts')
cutoff_type='bcmi'
directory = "/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/varandnonlinear_indices/seed" + str(seed_number)
if not os.path.exists(directory):
    os.makedirs(directory)
for i in var_cutoff:
    for j in bcmi_cutoff:
        for k in odc_cutoff:
            print("For ODC cutoff", k)
            print("For BCMI cutoff", j)
            print("For variance cutoff", i)
            # var causal and inpendent are the same because only the cis genes matter here 
            var_causal_list=pd.read_csv("./var_indices/yeast_wilko_var_causal_"+str(i)+".csv",header=None)[0].values.tolist()
            nonlinear_causal_list=pd.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/mpmi_indices/seed"+str(seed_number)+ "/yeast_wilko_mi_causal_"+cutoff_type+str(j)+"_"+str(k)+".csv",header=None)[0].values.tolist()
            nonlinear_indep_list=pd.read_csv("/data/users/cs20s037/CITNonLinear/findr/findrfiles/scripts/mpmi_indices/seed"+str(seed_number)+"/yeast_wilko_mi_indep_"+cutoff_type+str(j)+"_"+str(k)+".csv",header=None)[0].values.tolist()
            
            common_causal_len=len(set(var_causal_list).intersection(set(nonlinear_causal_list))) 
            common_indep_len=len(set(var_causal_list).intersection(set(nonlinear_indep_list))) 
            print('Length of var causal dataset',len(var_causal_list))
            print('Length of nonlinear causal dataset',len(nonlinear_causal_list))
            print('Length of causal dataset',common_causal_len)
            print('Length of independent dataset',common_indep_len)


            common_causal_list=list(set(var_causal_list).intersection(set(nonlinear_causal_list)))
            common_indep_list=list(set(var_causal_list).intersection(set(nonlinear_indep_list)))
            random_classifier=common_causal_len/(common_causal_len+common_indep_len)

            # if i==0 and j==0.05:
            #     print("printing the diference ............. ")
            #     #print(nonlinear_causal_list)
            #     diff1 = list(set(nonlinear_causal_list) - set(common_causal_list))
            #     print("In list1 but not in list2:", diff1)
            #     print(set(nonlinear_indep_list))
            #     print(set(common_indep_list))


            print("variance cutoff ",i,"bcmi both cutoff ",j,"random classifier ",random_classifier )
                    # Write to file only if the path doesn't exist
            output_file_path = directory + "/yeast_causal_var" + str(i) + cutoff_type + str(j)+"_"+str(k)+".csv"
            if not os.path.exists(output_file_path):
                np.savetxt(output_file_path,common_causal_list , delimiter=",", fmt='% s')
            
            output_file_path = directory + "/yeast_indep_var" + str(i) + cutoff_type + str(j)+"_"+str(k)+".csv"
            if not os.path.exists(output_file_path):
                np.savetxt(output_file_path, common_indep_list, delimiter=",", fmt='% s')

            output_file_path = directory + "/yeast_other_var" + str(i) + cutoff_type + str(j)+"_"+str(k)+".csv"
            if not os.path.exists(output_file_path):
                np.savetxt(output_file_path, [common_causal_len,common_indep_len,random_classifier], delimiter=",", fmt='% s')



For ODC cutoff 0.05
For BCMI cutoff 0.05
For variance cutoff 0
Length of var causal dataset 1752
Length of nonlinear causal dataset 63
Length of causal dataset 63
Length of independent dataset 63
printing the diference ............. 
In list1 but not in list2: []
{512, 384, 258, 1156, 1414, 1031, 136, 1033, 394, 908, 1038, 1168, 145, 1555, 1300, 1301, 1174, 1560, 1306, 1307, 539, 1437, 1312, 928, 1447, 1194, 1452, 941, 1711, 815, 1077, 1595, 1340, 444, 574, 447, 577, 1345, 1477, 72, 842, 846, 1231, 1360, 1491, 984, 986, 604, 1117, 863, 612, 484, 358, 1127, 1515, 108, 1645, 878, 753, 1523, 630, 1275, 1663}
{512, 384, 258, 1156, 1414, 1031, 136, 1033, 394, 908, 1038, 1168, 145, 1555, 1300, 1301, 1174, 1560, 1306, 1307, 539, 1437, 1312, 928, 1447, 1194, 1452, 941, 1711, 815, 1077, 1595, 1340, 444, 574, 447, 577, 1345, 1477, 72, 842, 846, 1231, 1360, 1491, 984, 986, 604, 1117, 863, 612, 484, 358, 1127, 1515, 108, 1645, 878, 753, 1523, 630, 1275, 1663}
variance cutoff  0 bcmi both cutoff  0